In [2]:
from google.colab import drive; drive.mount("/content/drive")

Mounted at /content/drive


In [18]:
import pandas as pd
import numpy as np
import nltk 
import re
nltk.download("stopwords");nltk.download("punkt")
import sklearn 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
data1 = pd.read_excel("/content/drive/My Drive/DimasASu/Data Latih BDC.xlsx")
data2 = pd.read_ csv("/content/drive/My Drive/DimasASu/data_clean.csv")

dataTest = pd.read_csv("/content/drive/My Drive/DimasASu/datatest_labelled.csv")

data1.head() , data2.head()

(    ID  ...  nama file gambar
 0   71  ...            71.jpg
 1  461  ...           461.png
 2  495  ...           495.png
 3  550  ...           550.png
 4  681  ...           681.jpg
 
 [5 rows x 6 columns],    Unnamed: 0  ... label
 0           0  ...     1
 1           1  ...     1
 2           2  ...     1
 3           3  ...     1
 4           4  ...     1
 
 [5 rows x 4 columns])

In [10]:
data1 = data1[["judul","narasi","label"]]
data2 = data2[["judul","narasi","label"]]

df_train = pd.concat([data1,data2])
df_train = df_train.reset_index().drop("index",axis = 1)
df_train.tail()

,judul,narasi,label
9658,foto raja salman mengangkat tandu jenazah korb...,foto raja salman mengangkat tandu jenazah korb...,1
9659,jokowi dituduh bodoh salah cara dalam bersalam...,jokowi dituduh bodoh salah cara dalam bersalam...,1
9660,jokowi dituduh memasukkan ribuan tenaga kerja ...,jokowi dituduh memasukkan ribuan tenaga kerja ...,1
9661,mata uang rupiah terpuruk,mata uang rupiah terpuruk,1
9662,ahok izinkan natalan di monas tapi larang peng...,ahok izinkan natalan di monas tapi larang peng...,1


In [29]:
def clean(text_messages):
    text_messages = str(text_messages)
    processed = re.sub(r'^.+@[^\.].*\.[a-z]{2,}$',
                                    'alamat email',text_messages)

    # Replace URLs with 'webaddress'
    processed = re.sub(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                      'alamat web',processed)

    # Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
    processed = re.sub(r'£|\$', 'moneysymb',processed)
        
    # Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
    processed = re.sub(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                      'nomor hp',processed)
        
    # Replace numbers with 'numbr'
    processed = re.sub(r'\d+(\.\d+)?', 'numbr',processed)




    # Remove punctuation
    processed = re.sub(r'[^\w\d\s]', ' ',processed)

    # Replace whitespace between terms with a single space
    processed = re.sub(r'\s+', ' ',processed)

    # Remove leading and trailing whitespace
    processed = re.sub(r'^\s+|\s+?$', '',processed)

    processed = processed.lower()

    return processed

In [31]:
df_train["narasi"] = df_train["narasi"].apply(clean)
df_train["judul"] = df_train["judul"].apply(clean)

In [33]:
from nltk.corpus import stopwords

stopwordsObj = set(stopwords.words("indonesian"))

df_train["narasi"] = df_train["narasi"].apply(lambda sentence: " ".join(word for word in sentence.split(" ") if word not in stopwordsObj))
df_train["judul"] = df_train["judul"].apply(lambda sentence: " ".join(word for word in sentence.split(" ") if word not in stopwordsObj))

In [35]:
porterStemmer = nltk.PorterStemmer()
df_train["narasi"] = df_train["narasi"].apply(lambda sentence: " ".join(porterStemmer.stem(word) for word in sentence.split(" ")) )

df_train["narasi"].head()

0    a caller to a radio talk show recent share tha...
1    yth anggota grup sesuai instruksi gubernur jaw...
2    jokowi presiden terbaik dlm sejarah bangsa ind...
3    maaf masnumbr mbaknumbr politik kenyataan joko...
4                   kadrun kalo lihat foto pana dingin
Name: narasi, dtype: object

In [36]:
from nltk.tokenize import word_tokenize

corpus = []

for message in df_train["narasi"]:
  words = word_tokenize(message)
  for word in words:
    corpus.append(word)

for message in df_train["judul"]:
  words = word_tokenize(message)
  for word in words:
    corpus.append(word)

In [38]:
corpus = nltk.FreqDist(corpus)
corpus

FreqDist({'a': 305,
          'caller': 2,
          'to': 241,
          'radio': 8,
          'talk': 5,
          'show': 14,
          'recent': 5,
          'share': 139,
          'that': 74,
          'hi': 29,
          'wife': 8,
          'wa': 177,
          'hospit': 19,
          'n': 79,
          'told': 6,
          'she': 19,
          'had': 4,
          'covid': 861,
          'onli': 13,
          'coupl': 3,
          'of': 332,
          'day': 34,
          'left': 10,
          'live': 23,
          'doctor': 13,
          'friend': 6,
          'suggest': 2,
          'be': 61,
          'test': 72,
          'for': 125,
          'legionnair': 8,
          'diseas': 13,
          'becaus': 19,
          'wore': 2,
          'the': 594,
          'same': 8,
          'mask': 39,
          'everi': 7,
          'all': 159,
          'long': 8,
          'turn': 18,
          'out': 35,
          'it': 126,
          'from': 71,
          'moistur': 3,
          

In [40]:
word_features = list(corpus.keys())[:5000]

In [58]:
def find_features(message):
  message = str(message)
  words = word_tokenize(message)
  features = {}
  for word in word_features:
    features[word] = word in words
  
  return features

In [76]:
messages = list(zip(df_train["narasi"],df_train["label"]))
np.random.seed(1)
print(messages)
featuresets = [(find_features(text),label) for (text,label) in messages] 

[('a caller to a radio talk show recent share that hi wife wa hospit n told she had covid n onli a coupl of day left to live a doctor friend suggest she be test for legionnair diseas becaus she wore the same mask everi day all day long turn out it wa legionnair diseas from the moistur n bacteria in her mask she wa given antibiot n within two day wa better what if these spike in covid are realli someth els due to mask induc infect', 1), ('yth anggota grup sesuai instruksi gubernur jawa hasil rapat tim gugu tuga covid numbr jateng sbb', 1), ('jokowi presiden terbaik dlm sejarah bangsa indonesia jim rohn motiv terbaik dunia', 1), ('maaf masnumbr mbaknumbr politik kenyataan jokowi berhasil memulangkan numbr numbr triliun uang negara swiss numbrribu triliun bawa indonesia ruu treati on mutual legal assist in crimin matter between the republ of indonesia and the swiss confeder mla ri swiss resmi disahkan dpr ri tanggal numbr juli numbr prose konstitusi menarik dana swiss selesai perjuangan m

In [73]:
featuresets[:5]

[(({'a': True,
    'caller': True,
    'to': True,
    'radio': True,
    'talk': True,
    'show': True,
    'recent': True,
    'share': True,
    'that': True,
    'hi': True,
    'wife': True,
    'wa': True,
    'hospit': True,
    'n': True,
    'told': True,
    'she': True,
    'had': True,
    'covid': True,
    'onli': True,
    'coupl': True,
    'of': True,
    'day': True,
    'left': True,
    'live': True,
    'doctor': True,
    'friend': True,
    'suggest': True,
    'be': True,
    'test': True,
    'for': True,
    'legionnair': True,
    'diseas': True,
    'becaus': True,
    'wore': True,
    'the': True,
    'same': True,
    'mask': True,
    'everi': True,
    'all': True,
    'long': True,
    'turn': True,
    'out': True,
    'it': True,
    'from': True,
    'moistur': True,
    'bacteria': True,
    'in': True,
    'her': True,
    'given': True,
    'antibiot': True,
    'within': True,
    'two': True,
    'better': True,
    'what': True,
    'if': Tru

In [77]:
from sklearn.model_selection import train_test_split
training,testing = train_test_split(featuresets,test_size = 0.05,random_state = 1)

print(len(training), len(testing))

9179 484


In [78]:
from nltk import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 83.47107438016529
Decision Tree Accuracy: 81.81818181818183
Random Forest Accuracy: 85.74380165289256
Logistic Regression Accuracy: 85.9504132231405
SGD Classifier Accuracy: 85.3305785123967
Naive Bayes Accuracy: 83.88429752066115
SVM Linear Accuracy: 85.3305785123967


In [85]:
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 85.3305785123967
